In [1]:
import gzip
from struct import unpack, pack, calcsize

In [13]:
# Offset + length_last - 1
# These values are taken from 
message_type_size_dict = {
    b"S" : 11,
    b"R" : 38,
    b"H" : 24,
    b"Y" : 19,
    b"L" : 25,
    b"V" : 34,
    b"W" : 11,
    b"K" : 27,
    b"J" : 34,
    b"h" : 20,
    b"A" : 35,
    b"F" : 39,
    b"E" : 30,
    b"C" : 35,
    b"X" : 22,
    b"D" : 18,
    b"U" : 34,
    b"P" : 43,
    b"Q" : 39,
    b"B" : 18,
    b"I" : 49,
    b"N" : 19,
    b"\x00" : 1
}

In [14]:
market_open='0930'
market_close = '1600'

In [43]:
def get_timestamp(binary_value):
    time_stamp = unpack('>6s', binary_value)[0];
    return int.from_bytes(time_stamp, byteorder='big')
    
def handle_system_messages():
    global market_o, market_s, market_q, market_m, market_e, market_c
    
    # System Message
    stock_locate, tracking_number = unpack('>HH', msg[0:4])
    event_code = unpack('>c', msg[10:])[0]
    time_stamp = get_timestamp(msg[4:10])
    if event_code == b"O":
        market_o = market_o + 1
        # Start of Messages.Outside oftime stamp messages,the start of day message isthe first message sentin
        # any trading day
    elif event_code == b"S":
        market_s = market_s + 1
        # Start of System hours. This message indicatesthatNASDAQis open and ready to start accepting orders
    elif event_code == b"Q":
        market_q = market_q + 1
        # Start of Market hours. This message isintended to indicate that Market Hours orders are available
        # for execution
    elif event_code == b"M":
        market_m = market_m + 1
        # End of Market hours. This message isintended to indicate that Market Hours orders are no longer
        # available for execution
    elif event_code == b"E":
        market_e = market_e + 1
        # End of System hours. It indicates that Nasdaq is now closed and will not accept any new orderstoday.
        # It is still possible to receive Broken Trade messages and Order Delete messages after the End of Day
    elif event_code == b"C":
        market_c = market_c + 1
        # End of Messages. This is always the last message sent in any trading day.
    print (stock_locate, tracking_number, time_stamp, event_code)

In [44]:
bin_data = open("20190130.PSX_ITCH_50", "rb")
message_type = bin_data.read(1);
market_o, market_s, market_q, market_m, market_e, market_c = (0, 0, 0, 0, 0, 0)
number_of_messages = 0
while message_type:
    msg = bin_data.read(message_type_size_dict[message_type])
    number_of_messages += 1
    # trading Message
    if message_type == b"H":
        temp = unpack('>HHhL8ccc4c', msg);
    elif message_type == b"S":
        handle_system_messages()
    message_type = bin_data.read(1);
    if number_of_messages > 1000000: # for all change to 100000000 or remove this if and break!
        break
    

print (number_of_messages, market_o, market_s, market_q, market_m, market_e, market_c)

0 0 11180838940054 b'O'
0 0 28800000025269 b'S'
0 0 34200000077413 b'Q'
1000001 1 1 1 0 0 0


In [ ]:
calcsize('>HHhL8ccc4c')
calcsize('>HHhLc')
calcsize('HHI')

# Making a 6 byte timestamp
def unpack48(x):
    x1, x2, x3 = struct.unpack('<HHI', x)
    return x1, x2 | (x3 << 16)
calcsize('HH6cc')

In [25]:
(1024).to_bytes(2, byteorder='big')

b'\x04\x00'